In [1]:
from create_spark import init_local_session, init_remote_session

/spark


In [1]:
import socket
import sys
import os
from pyspark import SparkContext, SparkConf
from pyspark.streaming import StreamingContext
from pyspark.sql import Row, SparkSession
from os.path import abspath
import findspark
import time 

In [ ]:
# # 스파크 생성 
# #spark = init_local_session()
# spark = SparkSession.builder.master('yarn').appName('SPARK_APP_NAME')\
#     .config("spark.sql.warehouse.dir", '/user/hive/warehouse')\
#     .config("spark.security.credentials.hiveserver2.enabled", 'false')\
#     .config("spark.sql.hive.hiveserver2.jdbc.url", 'jdbc:hive2://dn01:10000/metastore;user=hive;password=hive')\
#     .enableHiveSupport().getOrCreate()
#     #.config("spark.datasource.hive.warehouse.metastoreUri", 'thrift://localhost:9083')\
# # spark = init_remote_session(enable_hive=True)
# sc = spark.sparkContext
# sc

In [2]:
# 스파크 생성 
spark = SparkSession.builder.master('yarn').appName('SPARK_APP_NAME')\
    .config("spark.sql.warehouse.dir", '/user/hive/warehouse')\
    .getOrCreate()
    #.config("spark.datasource.hive.warehouse.metastoreUri", 'thrift://localhost:9083')\
# spark = init_remote_session(enable_hive=True)
sc = spark.sparkContext
sc

/usr/local/lib/python3.6/dist-packages/pyspark/context.py:238: FutureWarning: Python 3.6 support is deprecated in Spark 3.2.
  FutureWarning


<SparkContext master=yarn appName=SPARK_APP_NAME>

In [5]:
spark.read.text("hdfs://namenode:9000/user/hive/warehouse/testdb.db/intbl").show()

+-----+
|value|
+-----+
|  1a|
+-----+



In [4]:
#spark.sql("CREATE TABLE IF NOT EXISTS src (key INT, value STRING) USING hive") # Hive support is required
spark.sql("CREATE TABLE IF NOT EXISTS src (key INT, value STRING) ")

AnalysisException: Hive support is required to CREATE Hive TABLE (AS SELECT);
'CreateTable `default`.`src`, org.apache.hadoop.hive.serde2.lazy.LazySimpleSerDe, Ignore


In [5]:
spark.sql("select * from testdb.intbl;").show()

AnalysisException: Table or view not found: testdb.intbl; line 1 pos 14;
'Project [*]
+- 'UnresolvedRelation [testdb, intbl], [], false


In [6]:
spark.read.text("hdfs://namenode:9000/user/hive/warehouse/testdb.db/intbl").show()

+-----+
|value|
+-----+
|  1a|
+-----+



In [5]:
spark.stop()

NameError: name 'spark' is not defined

ERROR! Session/line number was not unique in database. History logging moved to new session 9
